In [1]:
import numpy as np
from sklearn.datasets import fetch_olivetti_faces
import pandas as pd
import plotly.express as px

In [2]:
class PCA:
    def __init__(self,n_components):
        self.n_components = n_components

    def fit(self,X):
        self.X_copy = X.copy().astype('float32')
        # do standardize the data
        self.mean_ = np.mean(self.X_copy, axis=0)
        self.std_ = np.std(self.X_copy, axis=0)
        self.X_copy -= self.mean_
        self.X_copy /= self.std_
        # calculate the covariance matrix and the eigen values/vectors
        cov_mat = np.cov(self.X_copy.T)    
        eigen_values, eigen_vectors = np.linalg.eigh(cov_mat)
        sorted_index = np.argsort(eigen_values)[::-1]
        self.sorted_eigenvalue_ = np.real(eigen_values[sorted_index])
        self.sorted_eigenvectors_ = np.real(eigen_vectors[:,sorted_index])
        self.explained_variance_ = self.sorted_eigenvalue_[0:self.n_components]
        total_var = self.sorted_eigenvalue_.sum()
        self.explained_variance_ratio_ = self.explained_variance_ / total_var
        self.cum_var_explained = np.cumsum(self.explained_variance_ratio_)

        self.projection_matrix_ = self.sorted_eigenvectors_[:,0:self.n_components]
        
    def transform(self,X,y):
        self.components_ = np.dot(self.X_copy,self.projection_matrix_)
        self.df = pd.DataFrame(data=self.components_, columns=['PC{}'.format(i+1) for i in range(self.n_components)])
        self.df['label'] = y
        return self.components_
    
    def fit_transform(self,X,y):
        self.fit(X)
        self.components_ = np.dot(self.X_copy,self.projection_matrix_)
        self.df = pd.DataFrame(data=self.components_, columns=['PC{}'.format(i+1) for i in range(self.n_components)])
        self.df['label'] = y
        return self.components_

    def pca_plot2d(self):
        fig = px.scatter(self.components_, x=0, y=1, color=self.df.label,labels={'0': 'PC 1', '1': 'PC 2'})
        fig.show()

    def pca_plot3d(self):
        fig = px.scatter_3d(self.components_, x=0, y=1,z=2, color=self.df.label,labels={'0': 'PC 1', '1': 'PC 2'})
        fig.show()    

# Load the dataset

In [3]:
ds = fetch_olivetti_faces()
faces = ds.data
images = ds.images

n_samples, width, height = images.shape
print("Dataset consists of {n} {w}x{h} faces".format(n=n_samples, w=width, h=height))
print('No. of classes:', np.unique( ds.target ).shape[0])

downloading Olivetti faces from https://ndownloader.figshare.com/files/5976027 to /root/scikit_learn_data
Dataset consists of 400 64x64 faces
No. of classes: 40


# Perform PCA on the dataset with several various n_components

In [4]:
list_components= range(1,100)
for c in list_components:
  pca = PCA(n_components=c)
  components = pca.fit_transform(faces, ds.target)
  total_var = pca.explained_variance_ratio_.sum() * 100
  print('{} components --> {} % cumulated variance explained'.format(c,total_var))

1 components --> 26.876448450582625 % cumulated variance explained
2 components --> 39.21882497740198 % cumulated variance explained
3 components --> 47.06023056770136 % cumulated variance explained
4 components --> 51.74882275588478 % cumulated variance explained
5 components --> 54.99503747433475 % cumulated variance explained
6 components --> 58.09581303661807 % cumulated variance explained
7 components --> 60.47964671593461 % cumulated variance explained
8 components --> 62.55136865343698 % cumulated variance explained
9 components --> 64.40923101777196 % cumulated variance explained
10 components --> 66.05543897724819 % cumulated variance explained
11 components --> 67.58959022807343 % cumulated variance explained
12 components --> 68.94327769271847 % cumulated variance explained
13 components --> 70.17684114720637 % cumulated variance explained
14 components --> 71.32076189731364 % cumulated variance explained
15 components --> 72.41513376745064 % cumulated variance explained
16 

# Plot the cumulated variance vs. n_component

In [5]:
pca = PCA(n_components=100)
components = pca.fit_transform(faces, ds.target)
total_var = pca.explained_variance_ratio_.sum() * 100

exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)

px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "# Components", "y": "Explained Variance"}
)

In [6]:
pca.pca_plot2d()

In [7]:
pca.pca_plot3d()